### osu!nn #6: Rhythm Predictor

Calculates a map's rhythm from the music and the timing.

Synthesis of "rhythmData"
* rhythmModel x 1
* momentumModel x 1
* timingData x 1
* (Music) x 1

Synthesis Time: ~2 seconds

Final edit: 2018/8/16

In [58]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, re
from config import *
from predict_CRNN import predict_CRNNs, read_npz_p

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
osuFile = "maps/317749 EYE_XY feat. Yoneko - Knight of Firmament/EYE_XY feat. Yoneko - Knight of Firmament (Pho) [Normal].osu"
model_1 = 'models/2019121/ConvLstm_Hard_batch1_lr0.001/model'
model_2 = 'models/2019121/ConvLstm2_Hard_batch1_lr0.001/model'

In [69]:
test_predictions, momentum_predictions_output, div_data2 = predict_CRNNs(osuFile, model_1, model_2)

fn = "mapthis.npz";
test_data, div_data, ticks, timestamps = read_npz_p(fn);

preds = test_predictions.reshape(-1, test_predictions.shape[2]);

# Favor sliders a little
preds[:, 2] += slider_favor;
divs = div_data2.reshape(-1, div_data2.shape[2]);
margin = np.sum([divisor_favor[k] * divs[:, k] for k in range(0, divisor)]);

preds[:, 0] += margin;

# Predict is_obj using note_density
obj_preds = preds[:, 0];
target_count = np.round(note_density[level] * obj_preds.shape[0]).astype(int);
print(target_count)
borderline = np.sort(obj_preds)[obj_preds.shape - target_count];
print(borderline)
is_obj_pred = np.expand_dims(np.where(preds[:, 0] > borderline, 1, 0), axis=1);

obj_type_pred = np.sign(preds[:, 1:4] - np.tile(np.expand_dims(np.max(preds[:, 1:4], axis=1), 1), (1, 3))) + 1;
others_pred = (1 + np.sign(preds[:, 4:test_predictions.shape[1]] + 0.5)) / 2;
another_pred_result = np.concatenate([is_obj_pred, is_obj_pred * obj_type_pred, others_pred], axis=1);

print("{} notes predicted.".format(np.sum(is_obj_pred)));

Map data saved! time = 1.0143952369689941 secs.
88
422
[-0.66628945]
421 notes predicted.


In [70]:
another_pred_result

array([[1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 1., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 1., 0., 0., 1., 0.]])

In [71]:
np.sum(another_pred_result[:,0])

421.0

In [72]:
momentum_predictions = momentum_predictions_output.reshape(-1, 2) * train_glob_std + train_glob_mean

In [73]:
momentum_predictions_output

array([[[-1.01526134e-01,  1.84719935e-02],
        [-5.54305375e-01, -5.41866571e-03],
        [-5.31912863e-01, -2.84941867e-03],
        ...,
        [-4.12073135e-01, -1.02596879e-02],
        [-5.19565880e-01,  9.73902643e-05],
        [-1.02840221e+00, -3.12030055e-02]],

       [[-1.01526134e-01,  1.84719935e-02],
        [-5.54305375e-01, -5.41866571e-03],
        [-5.31912863e-01, -2.84941867e-03],
        ...,
        [-4.12073135e-01, -1.02596879e-02],
        [-5.19565880e-01,  9.73902643e-05],
        [-1.02840221e+00, -3.12030055e-02]],

       [[-1.01526134e-01,  1.84719935e-02],
        [-5.54305375e-01, -5.41866571e-03],
        [-5.31912863e-01, -2.84941867e-03],
        ...,
        [-4.12073135e-01, -1.02596879e-02],
        [-5.19565880e-01,  9.73902643e-05],
        [-1.02840221e+00, -3.12030055e-02]],

       ...,

       [[ 4.58248965e-02,  1.76476985e-02],
        [-2.98731685e-01,  4.49303910e-03],
        [-4.18564528e-01,  1.88109651e-03],
        ...,
     

Save the rhythm data and progress to #7.

In [74]:
np.savez_compressed("rhythm_data", objs = is_obj_pred[:, 0], predictions = another_pred_result, timestamps = timestamps, ticks = ticks, momenta = momentum_predictions, sv = (div_data[:,6] + 1) * 150, dist_multiplier = dist_multiplier);

In [75]:
import json

rhythm_json = {
    "objs": is_obj_pred[:, 0].tolist(), 
    "predictions": another_pred_result.tolist(),
    "timestamps": timestamps.tolist(),
    "ticks": ticks.tolist(),
    "momenta": momentum_predictions.tolist(),
    "sv": ((div_data[:,6] + 1) * 150).tolist(),
    "distMultiplier": dist_multiplier
};
with open("evaluatedRhythm.json", "w") as er:
    json.dump(rhythm_json, er);